In [2]:
import sys
import time
from pyscf import gto, scf, mcscf, mp, cc, dft
from pyscf import lo
from pyscf.tools import molden, cubegen
from pyscf.geomopt.geometric_solver import optimize
import numpy
from pymatgen.core import Lattice, Structure, Molecule
from pyscf.dft import libxc
import pyscf


In [3]:
#Define the molecule settings (Coordinates, Basis, Spin, Charge, Unit)
mol = gto.Mole()
mol.fromfile("Ethene_Monomer.xyz") 
mol.basis = 'sto3g'
mol.symmetry = True
mol.spin = 0
mol.charge = 0
mol.unit = 'A'
mol.verbose = 5
mol.output = '/Users/andres/Desktop/out.out'
mol.build()
mol.atom_symbol


ValueError: invalid literal for int() with base 10: 'C         -0.66720        0.00000        0.00000'

In [361]:
#Use RKS , select the XC , Print Orbitals, Energies, Dipole ...
mf = dft.RKS(mol)
mf.xc = 'PBE'
#mf.xc = 'pbe,pbe'
#mf.xc = 'camb3lyp'
mf.kernel()
with open('mo_DFT.molden', 'w') as f1:
        molden.header(mol, f1) 
        molden.orbital_coeff(mol, f1, mf.mo_coeff, ene=mf.mo_energy, occ=mf.mo_occ)

print('Dipole moment %s' %(mf.dip_moment()))
print('Energy (a.u.) %5.6f' %(mf.e_tot))

print(mf.mo_coeff[0])
print(mf.mo_coeff[1])

Dipole moment [ 2.79908185e-05  8.33372720e-06 -6.35121514e-07]
Energy (a.u.) -77.507658
[-6.99362925e-01  7.01092165e-01 -1.80547390e-01  1.41290840e-01
 -9.51612060e-06 -1.70842883e-02  3.66493604e-06  0.00000000e+00
  0.00000000e+00 -9.32866156e-05 -1.26327644e-01 -1.66262367e-01
  3.89477660e-07 -1.20543684e-01]
[-4.05312998e-02  2.48278712e-02  4.72346345e-01 -4.29582314e-01
  3.42852489e-05  2.64373862e-02 -1.70368505e-05  0.00000000e+00
  0.00000000e+00  5.62401340e-04  8.19936353e-01  1.02214796e+00
 -4.40560496e-06  8.93794306e-01]


In [362]:
mf.init_guess

'minao'

In [363]:
#Geometry optmization, Convergence criteria can be selected ; For geomeTRIC contraints can be used (2 Molecules fixed distance)
#mol_eq = optimize(mf, maxsteps=100)

conv_params = { # These are the default settings
    'convergence_energy': 1e-6,  # Eh
    'convergence_grms': 3e-4,    # Eh/Bohr
    'convergence_gmax': 4.5e-4,  # Eh/Bohr
    'convergence_drms': 1.2e-3,  # Angstrom
    'convergence_dmax': 1.8e-3,  # Angstrom
}
mol_eq = optimize(mf, **conv_params)
print(mol_eq.atom_coords())
mol_eq.tofile('geoopt.xyz')
    


                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **              ********      .))))))  
          ##      .%%%%%%/      (%%%%%%,                  ,******      /)))))  
        %%%%%%      .%%%%%%#      *%%%%%%,    ,/////.       ******      )))))) 
      #%      %%      .%%%%%%/      *%%

'6\nXYZ from PySCF\nC          -0.67006        0.00000       -0.00000\nC           0.67006       -0.00000        0.00000\nH          -1.25419       -0.93260        0.07107\nH          -1.25418        0.93262       -0.07108\nH           1.25419        0.93260       -0.07107\nH           1.25418       -0.93262        0.07108'

In [366]:
molnew = dft.RKS(mol_eq)
molnew.xc = 'PBE'
molnew.kernel()
molnew.analyze()


((array([1.999599  , 1.05386367, 1.02568952, 1.02499287, 1.00014516,
         1.999599  , 1.05386367, 1.02568952, 1.02499287, 1.00014516,
         0.94785489, 0.9478549 , 0.94785489, 0.9478549 ]),
  array([-0.10429022, -0.10429022,  0.05214511,  0.0521451 ,  0.05214511,
          0.0521451 ])),
 array([ 1.01037333e-11,  3.11203254e-14, -2.36940845e-15]))

In [1]:
from qiskit_nature.drivers import PySCFDriver, UnitsType, QMolecule, Molecule
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
import numpy as np

atomtemp = mol.atom

#To convert from the mol object from PySCF into the format of the PySCFDriver
my_strings = []
for tupl in atomtemp:
    my_strings.append(
        f"{tupl[0]} {tupl[1][0]} {tupl[1][1]} {tupl[1][2]}"
    )
my_lst_str = ' ; '.join(my_strings)


driver = PySCFDriver(atom=my_lst_str,
                     unit=UnitsType.ANGSTROM,
                     basis='sto-3g')
problem = ElectronicStructureProblem(driver)
second_q_ops = problem.second_q_ops()



molecule = Molecule(geometry=mol.atom,charge=0, multiplicity=1)
driver = PySCFDriver(molecule = molecule, unit=UnitsType.ANGSTROM, basis='sto3g')
problem = ElectronicStructureProblem(driver)
second_q_ops = problem.second_q_ops()

NameError: name 'mol' is not defined

In [472]:
my_list_of_tuple = [
    ('C', [0, 1, 2]),
    ('H', [1, 1, 2]),
]
my_strings = []
for tupl in my_list_of_tuple:
    my_strings.append(
        f"{tupl[0]} {tupl[1][0]:.2f} {tupl[1][1]} {tupl[1][2]}"
    )

print(my_strings)


['C 0.00 1 2', 'H 1.00 1 2']
